<a href="https://colab.research.google.com/github/Rujjul/Synthetic_Dataset_Generator/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Synthetic_Dataset_Generator

In [3]:
!pip install -q --upgrade bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.7 MB/s eta 0:00:00


In [4]:
#imports
import os
import requests
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import gradio as gr

In [5]:
#LLM Model

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [6]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
quant_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16,
bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)


tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

In [7]:
system_message = """
You are a helper assistant who generates synthetic data. The user will provide you the column name, data type, number of rows, other parameters and constraints.Do not provide a python script to generate the data. Provide the data as a json with arrays.
"""

user_prompt = f"""
Create a synthetic dataset for an offical document.
Column names and type-
Name: not more than 13 alphabets
Phone Number: 10 digit number
Age: not more than 2 digit number
Occupation: 30 alphabets
PAN ID: 12 alphanumeric characters
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]
